# XGB Classifier with Filter-Based FS

Steven Sison | March 9, 2024

## Description

This document will be used to train a model using the reduced feature set obtain by using the wrapper-based method, forward feature selection. The model will be evaluated in terms of the usual metrics (accuracy, precision, F1-score, recall) as well as the training time. The model will also be stored for future evaluation purposes.

## Training the Model

### Preliminaries

#### 1. Loading the Dataset

In [3]:
import pandas as pd                     # For data transformation
import numpy as numpy                   # For scientific calculations
import seaborn as sns                   # For data visualizations
import matplotlib.pyplot as plt         # For plotting
import plotly.graph_objects as go       # For plotting
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, ConfusionMatrixDisplay
from xgboost import XGBClassifier, DMatrix, train
from sklearn.pipeline import Pipeline
import time
from datetime import datetime
import joblib
import os
import optuna
from sklearn.metrics import mean_squared_error # or any other metric
from sklearn.model_selection import train_test_split

dataset = pd.read_csv("../../../02_feature-engineering/final-datasets/binary_unbalanced_with_lexicalcontent.csv")      # Loading the dataset

dataset.head()

,url_type,blank_lines_count,blank_spaces_count,word_count,average_word_len,webpage_size,webpage_entropy,js_count,sus_js_count,js_eval_count,...,has_swf_in_string,has_cgi_in_string,has_php_in_string,has_abuse_in_string,has_admin_in_string,has_bin_in_string,has_personal_in_string,has_update_in_string,has_verification_in_string,url_scheme
0,1,14,36,0,0.000000,1186,5.269303,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,415,3952,11450,4.683930,26155,4.565537,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,5715,88450,420,5.833333,339327,4.656704,13,10,1,...,0,0,0,0,0,0,0,0,0,0
3,0,1461,6292,3049,4.515907,89249,5.149580,11,2,0,...,0,0,0,0,0,0,0,0,0,0
4,0,43,273,0,0.000000,1530,4.754726,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
dataset.head()

,url_type,blank_lines_count,blank_spaces_count,word_count,average_word_len,webpage_size,webpage_entropy,js_count,sus_js_count,js_eval_count,...,has_swf_in_string,has_cgi_in_string,has_php_in_string,has_abuse_in_string,has_admin_in_string,has_bin_in_string,has_personal_in_string,has_update_in_string,has_verification_in_string,url_scheme
0,1,14,36,0,0.000000,1186,5.269303,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,415,3952,11450,4.683930,26155,4.565537,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,5715,88450,420,5.833333,339327,4.656704,13,10,1,...,0,0,0,0,0,0,0,0,0,0
3,0,1461,6292,3049,4.515907,89249,5.149580,11,2,0,...,0,0,0,0,0,0,0,0,0,0
4,0,43,273,0,0.000000,1530,4.754726,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
x_train, x_test, y_train, y_test = train_test_split(dataset.drop(columns=['url_type']), dataset['url_type'], test_size = 0.2, random_state=42)

#### 2. Preprocessing (Balancing)

In [6]:
dataset['url_type'].value_counts()

url_type
0    120080
1     62334
Name: count, dtype: int64

#### 3. Removing Unnecessary Features

In [7]:
important_features_lexical_content = ['webpage_entropy',
                              'js_count', 
                              'js_search_count', 
                              'meta_tag_count', 
                              'has_log_in_html', 
                              'url_query_length', 
                              'url_path_length', 
                              'url_host_length', 
                              'url_num_subdomain', 
                              'url_num_periods', 
                              'url_num_forward_slash']

X_test_lexical_content = x_test[important_features_lexical_content]
X_train_lexical_content = x_train[important_features_lexical_content]

print("Lexical+Content-Based XGB Model has a total of "+str(len(important_features_lexical_content))+" features.")

Lexical+Content-Based XGB Model has a total of 11 features.


#### Hyper-parameter Optimization

In [8]:
# Define the objective function for Optuna
def objective_lexical_content(trial):
    # Define the search space for hyperparameters
    param = {
        'objective': 'binary:hinge',
        'eval_metric': 'error',
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'n_estimators': 100000, # Fix the boosting round and use early stopping
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 10.0),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.1, 10.0),
        'lambda': trial.suggest_float('lambda', 0.1, 10.0),
        'alpha': trial.suggest_float('alpha', 0.0, 10.0),
    }
    
    # Split the data into further training and validation sets (three sets are preferable)
    train_data, valid_data, train_target, valid_target = train_test_split(X_train_lexical_content, y_train, test_size=0.2, random_state=42)
    
    # Convert the data into DMatrix format
    dtrain = DMatrix(train_data, label=train_target)
    dvalid = DMatrix(valid_data, label=valid_target)
    
    # Define the pruning callback for early stopping
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'validation-error')
    
    # Train the model with early stopping
    model = train(param, dtrain, num_boost_round=100000, evals=[(dvalid, 'validation')], early_stopping_rounds=100, callbacks=[pruning_callback])
    
    # Make predictions on the test set
    dtest = DMatrix(valid_data)
    y_pred = model.predict(dtest)
    
    # Calculate the root mean squared error
    error = mean_squared_error(valid_target, y_pred, squared=False)
    
    return error

# Create an Optuna study and optimize the objective function
study_lexical_content = optuna.create_study(direction='minimize')
study_lexical_content.optimize(objective_lexical_content, n_trials=100) # Control the number of trials

# Print the best hyperparameters and the best RMSE
best_params_lexical_content = study_lexical_content.best_params
best_error_lexical_content = study_lexical_content.best_value
print("Best Hyperparameters: ", best_params_lexical_content)
print("Best Error: ", best_error_lexical_content)

[I 2024-04-09 10:31:29,252] A new study created in memory with name: no-name-428c0b06-6cd5-42c7-a523-b86de05d5ecb


[0]	validation-error:0.65995
[1]	validation-error:0.65995
[2]	validation-error:0.65995
[3]	validation-error:0.32809
[4]	validation-error:0.19036
[5]	validation-error:0.12266
[6]	validation-error:0.11279
[7]	validation-error:0.10875
[8]	validation-error:0.10282
[9]	validation-error:0.09929
[10]	validation-error:0.09155
[11]	validation-error:0.09011
[12]	validation-error:0.08812
[13]	validation-error:0.08764
[14]	validation-error:0.08788
[15]	validation-error:0.08679
[16]	validation-error:0.08624
[17]	validation-error:0.08562
[18]	validation-error:0.08528
[19]	validation-error:0.08545
[20]	validation-error:0.08507
[21]	validation-error:0.08463
[22]	validation-error:0.08380
[23]	validation-error:0.08295
[24]	validation-error:0.08261


c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [10:31:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[25]	validation-error:0.08254
[26]	validation-error:0.08237
[27]	validation-error:0.08226
[28]	validation-error:0.08154
[29]	validation-error:0.08106
[30]	validation-error:0.07928
[31]	validation-error:0.07925
[32]	validation-error:0.07928
[33]	validation-error:0.07846
[34]	validation-error:0.07815
[35]	validation-error:0.07747
[36]	validation-error:0.07678
[37]	validation-error:0.07675
[38]	validation-error:0.07654
[39]	validation-error:0.07531
[40]	validation-error:0.07469
[41]	validation-error:0.07414
[42]	validation-error:0.07322
[43]	validation-error:0.07212
[44]	validation-error:0.07161
[45]	validation-error:0.07037
[46]	validation-error:0.06965
[47]	validation-error:0.06787
[48]	validation-error:0.06667
[49]	validation-error:0.06667
[50]	validation-error:0.06616
[51]	validation-error:0.06578
[52]	validation-error:0.06503
[53]	validation-error:0.06421
[54]	validation-error:0.06438
[55]	validation-error:0.06393
[56]	validation-error:0.06397
[57]	validation-error:0.06287
[58]	valid

[I 2024-04-09 10:31:33,169] Trial 0 finished with value: 0.23501078311651893 and parameters: {'eta': 0.17705745252821742, 'max_depth': 4, 'subsample': 0.5657257492424042, 'colsample_bytree': 0.8430488199976606, 'gamma': 8.975994386296179, 'min_child_weight': 3.006209556543745, 'lambda': 2.9145461075585377, 'alpha': 8.59560265977502}. Best is trial 0 with value: 0.23501078311651893.


...

[0]	validation-error:0.65995
[1]	validation-error:0.41501
[2]	validation-error:0.12687
[3]	validation-error:0.07640
[4]	validation-error:0.06188
[5]	validation-error:0.05828
[6]	validation-error:0.05746
[7]	validation-error:0.05533
[8]	validation-error:0.05403
[9]	validation-error:0.05263
[10]	validation-error:0.05215
[11]	validation-error:0.05122
[12]	validation-error:0.05036
[13]	validation-error:0.04930
[14]	validation-error:0.04862
[15]	validation-error:0.04862
[16]	validation-error:0.04776
[17]	validation-error:0.04725
[18]	validation-error:0.04728
[19]	validation-error:0.04697
[20]	validation-error:0.04666
[21]	validation-error:0.04598
[22]	validation-error:0.04595
[23]	validation-error:0.04605
[24]	validation-error:0.04601
[25]	validation-error:0.04560
[26]	validation-error:0.04509
[27]	validation-error:0.04468
[28]	validation-error:0.04468
[29]	validation-error:0.04454
[30]	validation-error:0.04451
[31]	validation-error:0.04451
[32]	validation-error:0.04433
[33]	validation-erro

[I 2024-04-09 10:33:23,576] Trial 98 finished with value: 0.20377766468008437 and parameters: {'eta': 0.2661800926870541, 'max_depth': 10, 'subsample': 0.9908648399482195, 'colsample_bytree': 0.798000336441528, 'gamma': 0.23597576261467335, 'min_child_weight': 5.268672171023228, 'lambda': 9.105771077715012, 'alpha': 6.5894210993835625}. Best is trial 55 with value: 0.2017499757842728.


[0]	validation-error:0.65995
[1]	validation-error:0.65995


c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [10:33:23] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-04-09 10:33:23,769] Trial 99 pruned. Trial was pruned at iteration 2.


Best Hyperparameters:  {'eta': 0.2403652761559322, 'max_depth': 10, 'subsample': 0.9169178785003337, 'colsample_bytree': 0.6242994281142593, 'gamma': 0.39162401975547906, 'min_child_weight': 4.840960444191288, 'lambda': 9.585903073007772, 'alpha': 4.782446118760105}
Best Error:  0.2017499757842728


#### Model Training

In [9]:
from sklearn.model_selection import KFold
from xgboost import DMatrix, train

# Initialize CV
cv = KFold(n_splits=10, shuffle=True, random_state=1)

best_params_lexical_content['objective'] = 'binary:hinge'
best_params_lexical_content['eval_metric'] = 'error'

# Convert the data into DMatrix format
lexical_content_train = DMatrix(X_train_lexical_content, label=y_train)
lexical_content_valid = DMatrix(X_test_lexical_content, label=y_test)

# Train the Model
xgb_classifier = train(best_params_lexical_content, lexical_content_train, num_boost_round=1000)
y_pred_lexical_content = xgb_classifier.predict(lexical_content_valid)

print("Model training done.")

Model training done.


In [10]:
# Classification Report
print(classification_report(y_test, y_pred_lexical_content))

              precision    recall  f1-score   support

           0       0.96      0.97      0.97     23942
           1       0.95      0.93      0.94     12541

    accuracy                           0.96     36483
   macro avg       0.96      0.95      0.95     36483
weighted avg       0.96      0.96      0.96     36483



#### Practical Evaluation

In [11]:
# Dumping the model
joblib.dump(xgb_classifier, 'xgb_filter_lexical-content.sav')

['xgb_filter_lexical-content.sav']

In [14]:
import content_generator_hybrid
import feature_generation_content_function_htmlin
import time

def xgb_predict_maliciousness(url):

    numerical_values = content_generator_hybrid.feature_generator(url)
    # print(numerical_values)
    numerical_values = DMatrix(numerical_values)

    match xgb_classifier.predict(numerical_values):
        case 0:
            return "Benign"
        case 1:
            return "Malware"
        case 2:
            return "Phishing"
        case 3:
            return "Defacement"

url = "www.facebook.com/"
print("Current URL: "+url)

print("------------- Hybrid (Lexical + Content) -------------")
for i in range(15):
    start = time.perf_counter()
    prediction = xgb_predict_maliciousness(url)
    end = time.perf_counter()
    print("Trial "+str(i))
    print(prediction)
    print(end-start)

Current URL: www.facebook.com/
------------- Hybrid (Lexical + Content) -------------
Trial 0
Benign
0.48024309996981174
Trial 1
Benign
0.4593983999802731
Trial 2
Benign
0.4531106000067666
Trial 3
Benign
0.6433813999756239
Trial 4
Benign
0.42260049999458715
Trial 5
Benign
0.424024100007955
Trial 6
Benign
0.392601800034754
Trial 7
Benign
0.6416648000013083
Trial 8
Benign
0.43239739997079596
Trial 9
Benign
0.39887240005191416
Trial 10
Benign
0.42732219997560605
Trial 11
Benign
0.585705199977383
Trial 12
Benign
0.41874699998879805
Trial 13
Benign
0.6309899000334553
Trial 14
Benign
0.4743010000092909


#### Evaluation

In [ ]:
'''# Confusion Matrix for 12 Features
cm_up = confusion_matrix(y_test, y_pred, labels=xgb_classifier.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix = cm_up, display_labels = xgb_classifier.classes_)
disp.plot()
plt.show()'''

In [ ]:
'''# Cross Validation Score
scores = cross_val_score(XGBClassifier(**params_gbm),
                        X_train, y_train, scoring='accuracy', cv=cv).mean()

print(scores)'''